In [1]:
# package imports
import torch
import huggingface
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, GPT2Model, GPT2Tokenizer, AutoModelForCausalLM,GPT2LMHeadModel, GPT2Config)
from datasets import load_dataset

In [2]:
#import validation dataset for evaluation. 
dataset = load_dataset('rotten_tomatoes',split='validation') #just for evaluating

In [3]:
#shape of dataset
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1066
})

In [4]:
#instantiate model

#create tokenizer with padding
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
pad_token = "<PAD>"
tokenizer.pad_token = pad_token
#tokenizer.set_padding(tokenizer.pad_token, pad_to_multiple_of=8)
config = GPT2Config.from_pretrained("gpt2", pad_token_id=tokenizer.pad_token_id)
model = AutoModelForSequenceClassification.from_pretrained("gpt2",config=config)
#config = GPT2Config.from_pretrained("gpt2", pad_token_id=tokenizer.pad_token_id)

# Instantiate the model with the updated configuration
#model = GPT2ForSequenceClassification.from_pretrained("gpt2", config=config)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
#tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [6]:
#create a tokenized dataset for evaluation
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [7]:
#creating input for model
inputs = tokenizer.encode_plus(
    tokenized_datasets['text'],
    add_special_tokens=True,
    max_length=128,  # Maximum sequence length
    padding="max_length",
    truncation=True,
    return_tensors="pt"  # Return PyTorch tensors
)

In [8]:
# Make prediction
with torch.no_grad():
    outputs = model(**inputs).logits
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    predicted_class = torch.argmax(probabilities)

# Display sentiment result
if predicted_class == 1:
    print(f"Sentiment: Positive ({probabilities[0][1] * 100:.2f}%)")
else:
    print(f"Sentiment: Negative ({probabilities[0][0] * 100:.2f}%)")

Sentiment: Negative (67.12%)


In [9]:
#dataset labels
labels = dataset["label"]

In [10]:
#load full dataset for testing
full_dataset = load_dataset('rotten_tomatoes')

In [11]:
# Create a DataLoader to efficiently process the data
data_loader = torch.utils.data.DataLoader(list(zip(inputs["input_ids"],inputs["attention_mask"], labels)),
batch_size=16, shuffle=False)

In [12]:
#evaluate model performance
from sklearn.metrics import accuracy_score

tokenized_dataset = full_dataset.map(lambda examples: tokenizer(examples["text"], padding=True, truncation=True))
                                     #, batched=True)

# Prepare the data for evaluation
eval_dataset = tokenized_dataset["test"].remove_columns(["text"]).rename_column("label", "labels")
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.eval()
predictions = []
for batch in torch.utils.data.DataLoader(eval_dataset):
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        predictions.extend(predicted_labels.cpu().numpy())

true_labels = eval_dataset["labels"].numpy()
accuracy = accuracy_score(true_labels, predictions)
print("The pretrained model accuracy is", round(accuracy*100,2),"%")

The pretrained model accuracy is 49.34 %


In [13]:
from peft import LoraConfig
config = LoraConfig()

In [14]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

C:\Users\felix\anaconda3\Lib\site-packages\peft\tuners\lora\layer.py:711: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [15]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,736,256 || trainable%: 0.236428452686603


In [16]:
#create a training dataset for PEFT model
new_dataset = full_dataset.map(lambda examples: tokenizer(examples["text"], padding=True, truncation=True))

In [17]:
new_dataset = new_dataset.rename_column("label", "labels")
new_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [18]:
#unfreeze model
for param in lora_model.parameters():
    param.requires_grad = True

In [19]:
import numpy as np
import torch.nn as nn
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

In [20]:
#def compute_metrics(eval_pred):
#    predictions, labels = eval_pred
#    predictions = np.argmax(predictions, axis=1)
#    predictions = torch.from_numpy(predictions)  # Convert predictions to tensor
#    labels = torch.from_numpy(labels).float()  # Convert labels to tensor
#    loss = nn.CrossEntropyLoss()(predictions, labels)  # Calculate the evaluation loss
#    accuracy = (predictions == labels).float().mean()  # Calculate the accuracy
#    return {"eval_loss": loss, "accuracy": accuracy}

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids

    # Convert predictions and labels to the appropriate data types
    predictions = torch.from_numpy(predictions).float()
    labels = torch.from_numpy(labels).long()

    loss = nn.CrossEntropyLoss()(predictions, labels)
    accuracy = (predictions.argmax(dim=1) == labels).float().mean()

    return {"eval_loss": loss, "accuracy": accuracy}

#def compute_metrics(eval_pred):
#    predictions = torch.from_numpy(eval_pred.predictions)  # Convert predictions to tensor
#    labels = torch.from_numpy(eval_pred.label_ids).float()  # Convert labels to tensor and cast to float
#    loss = nn.CrossEntropyLoss()(predictions, labels)  # Calculate the evaluation loss
#    accuracy = (predictions == labels).float().mean()  # Calculate the accuracy
#    return {"eval_loss": loss, "accuracy": accuracy}

In [21]:
import os

output_dir = os.path.join(os.path.expanduser("~"), "Documents", "output")
os.makedirs(output_dir, exist_ok=True)

In [22]:
training_args = TrainingArguments(output_dir=output_dir,
                                  evaluation_strategy='epoch', 
                                  save_strategy='epoch',
                                  learning_rate=.2, 
                                  per_device_train_batch_size=18,
                                  per_device_eval_batch_size=18,
                                  num_train_epochs=1, 
                                  load_best_model_at_end=True,
                                  weight_decay=0.1,
                                  remove_unused_columns=False,
                                  label_names=["labels"]
                                 )


In [ ]:
trainer = Trainer(
    model=lora_model,
    args = training_args,
    train_dataset = new_dataset['train'],
    eval_dataset = new_dataset['test'],
    tokenizer=tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics = compute_metrics
)
trainer.train()

Epoch,Training Loss,Validation Loss
